In [28]:
import numpy as np
import random

## Defining paramters for the SGA

In [29]:
pop_size = 10
chromosome_length = 5
generation_number = 1
crossover_rate = 0.5
mutation_rate = 0.1

## Defining the Individual

In [30]:
class Individual:
    def __init__(self, genotype) -> None:
        self.genotype = genotype
        self.phenotype = self.get_phenotype(genotype)
    
    def get_phenotype(self, genotype):
        return int(genotype, 2)

## Defining the population

In [31]:
population = []

## Sin Synthetic Problem

*a) Implement a function to generate an initial population for your genetic
algorithm.*

We are choosing to initialize the population, according to the size parameters. The actual genotypes (0's and 1's) will be generated randomly.

In [32]:
for _ in range(pop_size):
    potential_genotype = ""
    for _ in range(chromosome_length):
        potential_genotype += str(random.randint(0,1))
    population.append(potential_genotype)

*b) Implement a parent selection function for your genetic algorithm. This function
should find the fittest individuals in the population, and select parents based
on this fitness.*